In [1]:

from mpdaf.obj import Cube
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from astropy import units as u
from numpy import ma
import numpy as np
from astropy.table import Table, Column, MaskedColumn, pprint
from astropy.io import fits
from astropy.wcs import WCS
from mpdaf.obj import Image
from scipy import integrate


# Apply HST Bandpass Filter to MUSE Cube 

- To get mage and wcs with filter applied.
- There is choice 814 or 606

In [2]:
def bandpass_image(self, wavelengths, sensitivities, unit_wave=u.angstrom,
                       interpolation="linear"):
        """
        bandpass_image sums the images of a cube after multiplying the cube by a given spectral bandpass curve.
        
        
        Given a cube of images versus wavelength and the bandpass
        filter-curve of a wide-band monochromatic instrument, extract
        an image from the cube that has the spectral response of the
        monochromatic instrument.

        For example, this can be used to create a MUSE image that has
        the same spectral characteristics as an HST image. The MUSE
        image can then be compared to the HST image without having to
        worry about any differences caused by different spectral
        sensitivities.

        For each channel n of the cube, the filter-curve is integrated
        over the width of that channel to obtain a weight, w[n]. The
        output image is then given by the following weighted mean::

            output_image = sum(w[n] * cube_image[n]) / sum(w[n])

        In practice, to accommodate masked pixels, the w[n] array is
        expanded into a cube w[n,y,x], and the weights of individual
        masked pixels in the cube are zeroed before the above equation
        is applied.

        If the wavelength axis of the cube only partly overlaps the
        bandpass of the filter-curve, the filter curve is truncated to
        fit within the bounds of the wavelength axis. A warning is
        printed to stderr if this occurs, because this results in an
        image that lacks flux from some of the wavelengths of the
        requested bandpass.

        Parameters
        ----------
        wavelengths : numpy.ndarray
            An array of the wavelengths of the filter curve,
            listed in ascending order of wavelength. Outside
            the listed wavelengths the filter-curve is assumed
            to be zero.
        sensitivities : numpy.ndarray
            The relative flux sensitivities at the wavelengths
            in the wavelengths array. These sensititivies will be
            normalized, so only their relative values are important.
        unit_wave : `astropy.units.Unit`
            The units used in the array of wavelengths. The default is
            angstroms. To specify pixel units, pass None.
        interpolation : str
            The form of interpolation to use to integrate over the
            filter curve. This should be one of::

              "linear"     : Linear interpolation
              "cubic"      : Cubic spline interpolation (very slow)

            The default is linear interpolation. If the filter curve
            is well sampled and its sampling interval is narrower than
            the wavelength pixels of the cube, then this should be
            sufficient. Alternatively, if the sampling interval is
            significantly wider than the wavelength pixels of the
            cube, then cubic interpolation should be used instead.
            Beware that cubic interpolation is much slower than linear
            interpolation.

        Returns
        -------
        `~mpdaf.obj.Image`
            An image formed from the filter-weighted mean of channels in
            the cube that overlap the bandpass of the filter curve.

        """
        from scipy import integrate

        wavelengths = np.asarray(wavelengths, dtype=float)
        sensitivities = np.asarray(sensitivities, dtype=float)

        if (wavelengths.ndim != 1 or sensitivities.ndim != 1 or
                len(wavelengths) != len(sensitivities)):
            raise ValueError('The wavelengths and sensititivies arguments'
                             ' should be 1D arrays of equal length')

        if unit_wave is None:
            pixels = wavelengths.copy()
        else:
            pixels = self.wave.pixel(wavelengths, unit=unit_wave)

        # Get the integer indexes of the pixels that contain the above
        # floating point pixel indexes.
        print(pixels)
        indexes = np.rint(pixels).astype(int)
        print(indexes)

        # If there is no overlap between the bandpass filter curve
        # and the wavelength coverage of the cube, complain.
        if indexes[0] >= self.shape[0] or indexes[-1] < 0:
            raise ValueError("The filter curve does not overlap the "
                             "wavelength coverage of the cube.")

        # To correctly reproduce an image taken through a specified
        # filter, the bandpass curve should be completely encompassed
        # by the wavelength axis of the cube. If the overlap is
        # incomplete, emit a warning, then truncate the bandpass curve
        # to the edge of the wavelength range of the cube.
        if indexes[0] < 0 or indexes[-1] >= self.shape[0]:

            # Work out the start and stop indexes of the slice needed
            # to truncate the arrays of the bandpass filter curve.
            if indexes[0] < 0:
                start = np.searchsorted(indexes, 0, 'left')
            else:
                start = 0
            if indexes[-1] >= self.shape[0]:
                stop = np.searchsorted(indexes, self.shape[0], 'left')
            else:
                stop = indexes.shape[0]

            # Integrate the overal bandpass filter curve.
            total = integrate.trapz(sensitivities, wavelengths)

            # Also integrate over just the truncated parts of the curve.
            lost = 0.0
            if start > 0:
                s = slice(0, start)
                lost += integrate.trapz(sensitivities[s], wavelengths[s])
            if stop < indexes.shape[0]:
                s = slice(stop, indexes.shape[0])
                lost += integrate.trapz(sensitivities[s], wavelengths[s])

            # Compute the fraction of the integrated bandpass response
            # that has been truncated.
            lossage = lost / total

            # Truncate the bandpass filter curve.
            indexes = indexes[start:stop]
            pixels = pixels[start:stop]
            sensitivities = sensitivities[start:stop]

            # Report the loss if it is over 0.5%.
            if lossage > 0.005:
                self._logger.warning(
                    "%.2g%% of the integrated " % (lossage * 100.0) +
                    "filter curve is beyond the edges of the cube.")

        # Get the range of indexes along the wavelength axis that
        # encompass the filter bandpass within the cube.
        kmin = indexes[0]
        kmax = indexes[-1]
        print('Len(k):', kmax - kmin + 1) 
        print(start)
        print(stop)
        print('Subcube k start:', self.wave.coord(kmin))
        print('Subcube k stop:', self.wave.coord(kmax)) #  not sure this is actually how it is being used 
        
        


        # Obtain an interpolator of the bandpass curve.
        spline = interp1d(x=pixels, y=sensitivities,
                                      kind=interpolation)

        # Integrate the bandpass over the range of each spectral pixel
        # to determine the weights of each pixel. For the moment skip
        # the first and last pixels, which need special treatment.
        # Integer pixel indexes refer to the centers of pixels,
        # so for integer pixel index k, we need to integrate from
        # k-0.5 to k+0.5.
        w = np.empty((kmax + 1 - kmin))
        for k in range(kmin + 1, kmax):
            w[k - kmin], err = integrate.quad(spline, k - 0.5, k + 0.5)

        # Start the integration of the weight of the first channel
        # from the lower limit of the bandpass.
        w[0], err = integrate.quad(spline, pixels[0], kmin + 0.5)

        # End the integration of the weight of the final channel
        # at the upper limit of the bandpass.
        w[-1], err = integrate.quad(spline, kmax - 0.5, pixels[-1])

        # Normalize the weights.
        w /= w.sum()

        # Create a sub-cube of the selected channels.
        subcube = self[kmin:kmax + 1, :, :] 

        # To accommodate masked pixels, create a cube of the above
        # weights, but with masked pixels given zero weight.
        if subcube._mask is ma.nomask:
            wcube = w[:, np.newaxis, np.newaxis] * np.ones(subcube.shape)
        else:
            wcube = w[:, np.newaxis, np.newaxis] * ~subcube._mask

        # Get an image which is the sum of the weights along the spectral axis.
        wsum = wcube.sum(axis=0)

        # The output image is the weighted mean of the selected
        # channels. For each map pixel perform the following
        # calculation over spectral channels, k.
        #
        #  mean = sum(weights[k] * data[k]) / sum(weights[k]
        data = np.ma.sum(subcube.data * wcube, axis=0) / wsum

        # The variance of a weighted means is:
        #
        #  var = sum(weights[k]**2 * var[k]) / (sum(weights[k]))**2
        if subcube._var is not None:
            var = np.ma.sum(subcube.var * wcube**2, axis=0) / wsum**2
        else:
            var = False

        return Image.new_from_obj(subcube, data=data, var=var)


In [6]:
# data array is read from the file (extension number 0)
cube_26 = Cube(filename='/Users/s2537809/MUSE/cube_26.fits')

cube_27 = Cube(filename='/Users/s2537809/MUSE/cube_27.fits')

cube_28 = Cube(filename='/Users/s2537809/MUSE/cube_28.fits')

cube_UDF = Cube(filename='/Users/s2537809/MUSE/DATACUBE_UDF-10.fits')

# 606 or 814
with open('HST_ACS_HRC.F606W.dat', 'r') as file:
    HST = file.read()


# Split the data into lines
HST = HST.split('\n')

# Split each line into columns and filter out any lines that don't have 2 columns
rows = [row.split() for row in HST if len(row.split()) == 2]

HST_table = Table(rows=rows, names=['Wavelength', 'Throughput'])

HST_k = np.array(HST_table['Wavelength']).astype(float)
HST_w = np.array(HST_table['Throughput']).astype(float)



im26 = bandpass_image(cube_26, HST_table['Wavelength'], HST_table['Throughput'])
im27 = bandpass_image(cube_27, HST_table['Wavelength'], HST_table['Throughput']) ## Is the throughput actually good to use for sensitvities 
im28 = bandpass_image(cube_28, HST_table['Wavelength'], HST_table['Throughput'])
imUDF = bandpass_image(cube_UDF, HST_table['Wavelength'], HST_table['Throughput'])



# Save WCS and image data to FITS function
def save_wcs_to_fits(im_data, wcs_object, fits_filename):
    hdu = fits.PrimaryHDU(im_data.filled())
    hdu.header.extend(wcs_object.to_header(), update=True)
    hdu.writeto(fits_filename, overwrite=True)


# Example usage for each bandpass cube
save_wcs_to_fits(im26.data, im26.wcs, 'im26_wcs.fits')
save_wcs_to_fits(im27.data, im27.wcs, 'im27_wcs.fits')
save_wcs_to_fits(im28.data, im28.wcs, 'im28_wcs.fits')
save_wcs_to_fits(imUDF.data, imUDF.wcs, 'imUDF_wcs.fits')

/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:119: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  total = integrate.trapz(sensitivities, wavelengths)
/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:125: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  lost += integrate.trapz(sensitivities[s], wavelengths[s])


[-105.06171875 -104.26171875 -103.46171875 ... 2062.93828125 2063.73828125
 2064.53828125]
[-105 -104 -103 ... 2063 2064 2065]
Len(k): 2066
131
2713
Subcube k start: 4700.3271484375
Subcube k stop: 7281.5771484375
[-105.14765625 -104.34765625 -103.54765625 ... 2062.85234375 2063.65234375
 2064.45234375]
[-105 -104 -104 ... 2063 2064 2064]
Len(k): 2065
131
2713
Subcube k start: 4700.4345703125
Subcube k stop: 7280.4345703125


/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:119: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  total = integrate.trapz(sensitivities, wavelengths)
/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:125: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  lost += integrate.trapz(sensitivities[s], wavelengths[s])


[-105.14960938 -104.34960938 -103.54960938 ... 2062.85039063 2063.65039062
 2064.45039063]
[-105 -104 -104 ... 2063 2064 2064]
Len(k): 2065
131
2713
Subcube k start: 4700.43701171875
Subcube k stop: 7280.43701171875


/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:119: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  total = integrate.trapz(sensitivities, wavelengths)
/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:125: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  lost += integrate.trapz(sensitivities[s], wavelengths[s])
/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:119: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.trapezoid' and will be removed in SciPy 1.14.0
  total = integrate.trapz(sensitivities, wavelengths)
/var/folders/wq/x4d7t0dd705bwvpnmkf6s7s40000gn/T/ipykernel_94254/2089795275.py:125: DeprecationWarning: 'scipy.integrate.trapz' is deprecated in favour of 'scipy.integrate.tra

[-144.8 -144.  -143.2 ... 2023.2 2024.  2024.8]
[-145 -144 -143 ... 2023 2024 2025]
Len(k): 2026
181
2713
Subcube k start: 4750.0
Subcube k stop: 7281.25
